# VARS-TOOL Quick Start Tutorial

## Install

This is a simple tutorial show casing VARS-TOOL functionalities that are being implement in Python.Let's install the latest VARS-TOOL from GitHub:

In [1]:
!rm -rf vars-tool
!git clone https://github.com/vars-tool/vars-tool
!pip install vars-tool/.

Cloning into 'vars-tool'...
remote: Enumerating objects: 1759, done.
remote: Counting objects: 100% (489/489), done.
remote: Compressing objects: 100% (287/287), done.
remote: Total 1759 (delta 284), reused 373 (delta 178), pack-reused 1270
Receiving objects: 100% (1759/1759), 18.38 MiB | 16.75 MiB/s, done.
Resolving deltas: 100% (1076/1076), done.
Processing ./vars-tool
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for varstool: filename=varstool-2.1-py2.py3-none-any.whl size=429333 sha256=6167c0a09bc74ed696d6020384a9bf78b1a3ad3fc7e5f32d64524b29bc24078d
  Stored in directory: /home/jovyan/.cache/pip/wheels/d1/76/56/bddc63ebee22fd00eb32e22d073f5268b15c173aee473129fa
Successfully built varstool
  Attempting uninstall: varstool
    Found existing installation: varstool 2.1
    Uninstalling varstool-2.1:
      Successfully uninstalled varstool-2.1


## Example using the Ishigami and wavy6D models

Let's import VARS for the analysis and the Model class for creating a wrapper around the desired model so that it can be inputted into VARS

In [2]:
# importing VARS and Model from varstool package
from varstool import VARS, Model

# these are being imported in order for us to display the data nicely on the notebook
import numpy as np
import pandas as pd

Here is a quick function definition of the Ishigami model:

In [3]:
def ishigami(x, a=7, b=0.05):
    '''Ishigami test function'''
    # check whether the input x is a dataframe
    
    if not isinstance(x, (pd.core.frame.DataFrame, pd.core.series.Series, np.ndarray, list)):
        raise TypeError('`x` must be of type pandas.DataFrame, numpy.ndarray, pd.Series, or list')
    
    if len(x) > 3:
        raise ValueError('`x` must have only three arguments at a time')
    
    return np.sin(x[0]) + a*(np.sin(x[1])**2) + b*(x[2]**4)*np.sin(x[0])

Here is a quick function definition of the wavy6D model:

In [4]:
def wavy6D(x):
    '''wavy6D test function'''
    
# The features include:             
# (1) The 6 input factors are non-interacting                  
# (2) The function is "wavy" along the first 5 dimensions but with different 
#     frequencies and amplitude at different directions                                     
# (3) The last dimension (X6) is fully isensitive (dummy variable)                                         
# (4) It possess large-scale features such multi-modality (X1) 
#     to small-scale features such as roughness (X3)          
# (5) The directional variograms of X1 and X2 cross each other
    
    # check whether the input x is a dataframe
    
    if not isinstance(x, (pd.core.frame.DataFrame, pd.core.series.Series, np.ndarray, list)):
        raise TypeError('`x` must be of type pandas.DataFrame, numpy.ndarray, pd.Series, or list')
    
    if len(x) > 6:
        raise ValueError('`x` must have only five arguments at a time')
    
    y1 = -1*np.sin(2*np.pi*x[0]/2) - 0.3*np.sin(2*np.pi*x[0]/0.6)
    y2 = 0.76*(-1*np.sin(2*np.pi*(x[1]-0.2)/2)) - 0.315
    y3 = 0.12*(-1*np.sin(2*np.pi*(x[2]-0.2)/1.9)) + 0.02*(-1*np.sin(2*np.pi*x[2]/0.021)) - 0.96
    y4 = 0.12*(-1*np.sin(2*np.pi*(x[3]-0.2)/1.9))-0.97
    y5 = 0.05*(-1*np.sin(2*np.pi*(x[4]-0.2)/2))-1.02
    y6 = -1.08

    return y1 + y2 + y3 + y4 + y5 + y6

As mentioned previously The `Model` class is a wrapper for custom functions(models) in the online version of VARS. However, the first paramter of every function must accept an array of parameters

Here we will create two wrappers for the testing models ishigami and wavy6D:

In [5]:
ishigami_model = Model(ishigami)

In [6]:
wavy6D_model = Model(wavy6D)

When creating an experiment you will need to assign a variable to an instance of VARS as shown below.

The paramters of the VARS class are described as:

**paramaters**: the name of each paramter along with their upper and lower bounds

**num_stars**: the total number of star samples that are desired for STAR-VARS analysis

**delta_h**: the sampling resolution of the VARS analysis

**ivars_scales**: the scales that are to be used when doing IVARS, e.g, 0.1 and 0.3 correspond (0-0.1) and (0-0.3) <br /> note: can not have a scale larger then 0.5

**star_centres**: This is only used if a sampler is not chosen and you are wanting to generate your own star centres(randomized numbers)

**sampler**: the sampling strategy: `rnd`, `lhs`, `plhs`, `sobol_seq`, or `halton_seq` for generation of star centers.

**seed**: the seed number for randomization of the sampling strategy specified by `sampler`, only needed if a sampler was chosen <br /> note: seed is a randomized integer from 1 to 123456789 as default if no seed is chosen

**model**: the wrapper of your model you made when using the `Model` class

**bootstrap_flag**: this is a True/False value that specifies if bootstrapping will be used in the VARS analysis

**bootstrap_size**: the number of sampling iterations with replacement

**bootstrap_ci**: the bootstrap-based confidence interval on results

**grouping_flag**: this is a True/False value that specifies if paramater grouping will be used in the VARS analysis

**num_grps**: the number of groups you want to split your paramaters into, if left blank the optimal number of groups will be calculated by VARS

**report_verbose**: this is a True/False value that if True will display a loading bar to show the progession of the VARS analysis, else there will be no progression loading bar

Create `experiment_1` and `experiment_2` then initialize the values needed to run a VARS analysis:

`experiment_1` is an instance of VARS that is using the ishigami model

In [7]:
experiment_1 = VARS(parameters = {'x1':(0, 1), 'x2':(0, 1), 'x3': (0, 1)},
                    num_stars=10,
                    delta_h = 0.1,
                    ivars_scales = (0.1, 0.3, 0.5),
                    sampler = 'rnd',
                    seed = 123456789,
                    model = ishigami_model,
                    bootstrap_flag = True,
                    bootstrap_size = 100,
                    bootstrap_ci=0.9,
                    grouping_flag=True,
                    num_grps=2,
                    report_verbose=True,
                )

`experiment_2` is an instance of VARS that is using the wavy6D model

In [8]:
experiment_2 = VARS(parameters = {'x1':(0, 1), 'x2':(0, 1), 'x3': (0, 1), \
                                  'x4':(0, 1), 'x5':(0, 1), 'x6': (0, 1)},
                    num_stars=10,
                    delta_h = 0.1,
                    ivars_scales = (0.1, 0.3, 0.5),
                    sampler = 'rnd',
                    seed = 123456789,
                    model = wavy6D_model,
                    bootstrap_flag = True,
                    bootstrap_size = 100,
                    bootstrap_ci=0.9,
                    grouping_flag=True,
                    num_grps=2,
                    report_verbose=False,
                )

A report displaying the current status of the VARS analysis can be found by typing in the variable name of the instance you created, here this is `experiment_1`

In [9]:
experiment_1

Star Centres: 10 Centers Loaded
Star Points: Not Loaded
Parameters: 3 paremeters set
Delta h: 0.1
Model: ishigami
Seed Number: 123456789
Bootstrap: On
Bootstrap Size: 100
Bootstrap CI: 0.9
Grouping: On
Number of Groups: 2
VARS Analysis: Not Done

To run the VARS analysis we can simply do the following:

note: we are using `experiment_1` here, but `experiment_2` can be used in the same exact way

In [10]:
experiment_1.run_online()

function evaluation:   0%|          | 0/280 [00:00<?, ?it/s]

building pairs:   0%|          | 0/30 [00:00<?, ?it/s]

VARS Analysis:   0%|          | 0/10 [00:00<?, ?it/s]

factor ranking:   0%|          | 0/2 [00:00<?, ?it/s]

Bootstrapping:   0%|          | 0/100 [00:00<?, ?it/s]

Now if we take a look at the status report again we can see that the `Vars Analysis` is now done.

In [11]:
experiment_1

Star Centres: 10 Centers Loaded
Star Points: Loaded
Parameters: 3 paremeters set
Delta h: 0.1
Model: ishigami
Seed Number: 123456789
Bootstrap: On
Bootstrap Size: 100
Bootstrap CI: 0.9
Grouping: On
Number of Groups: 2
VARS Analysis: Done

Most useful results are stored in the `.output` property as a dictionary. The variables stores could be seen quickly by:

In [12]:
experiment_1.output.keys()

dict_keys(['Gamma', 'MAEE', 'MEE', 'COV', 'ECOV', 'IVARS', 'IVARSid', 'rnkST', 'rnkIVARS', 'Gammalb', 'Gammaub', 'STlb', 'STub', 'IVARSlb', 'IVARSub', 'relST', 'relIVARS', 'Groups', 'relGrp'])

and each output can be viewed by calling similar method like below:

In [13]:
experiment_1.output['Gamma'].unstack(level=0) # adding unstack to make it more legible

param,x1,x2,x3
h,,,
0.1,0.003698,0.149466,0.000007
0.2,0.014897,0.608676,0.000023
0.3,0.033664,1.381880,0.000041
0.4,0.059940,2.455592,0.000060
0.5,0.093541,3.797486,0.000080
0.6,0.134157,5.356835,0.000105
0.7,0.181351,7.066600,0.000138
0.8,0.234574,8.847068,0.000189
0.9,0.293164,10.610789,0.000269


Now we can access all the results using 'dot' notation on our created instance:

**Directional variogram results**

There are 10 rows as the number of stars was 10, and each at a resolution incremented by 0.1 which was specificed to be the sampling resolution

In [14]:
experiment_1.gamma.unstack(0)

param,x1,x2,x3
h,,,
0.1,0.003698,0.149466,0.000007
0.2,0.014897,0.608676,0.000023
0.3,0.033664,1.381880,0.000041
0.4,0.059940,2.455592,0.000060
0.5,0.093541,3.797486,0.000080
0.6,0.134157,5.356835,0.000105
0.7,0.181351,7.066600,0.000138
0.8,0.234574,8.847068,0.000189
0.9,0.293164,10.610789,0.000269


**Integrated variogram**

In [15]:
experiment_1.ivars

,x1,x2,x3
0.1,0.000185,0.007473,3.594623e-07
0.3,0.003543,0.144908,5.020546e-06
0.5,0.015897,0.649436,1.705942e-05


**VARS-TO: Sobol total-order effects calculated through VARS**

In [16]:
experiment_1.st

param
x1    0.027037
x2    1.084649
x3    0.000022
dtype: float64

**VARS-ABE: Morris mean absolute elementary effects across scales**

In [17]:
experiment_1.maee.unstack(0)

param,x1,x2,x3
h,,,
0.1,0.085066,0.511523,0.002353
0.2,0.171142,1.048732,0.004331
0.3,0.257799,1.601071,0.006094
0.4,0.344604,2.157385,0.007803
0.5,0.431119,2.706208,0.009619
0.6,0.516906,3.236077,0.011704
0.7,0.601532,3.735833,0.014218
0.8,0.684567,4.194928,0.017323
0.9,0.765593,4.603709,0.021178


**VARS-ACE: Morris mean actual elementary effects across scales**

In [18]:
experiment_1.mee.unstack(0)

param,x1,x2,x3
h,,,
0.1,0.085066,0.511523,0.002353
0.2,0.171142,1.048732,0.004331
0.3,0.257799,1.601071,0.006094
0.4,0.344604,2.157385,0.007803
0.5,0.431119,2.706208,0.009619
0.6,0.516906,3.236077,0.011704
0.7,0.601532,3.735833,0.014218
0.8,0.684567,4.194928,0.017323
0.9,0.765593,4.603709,0.021178


**The factor(parameter) rankings based on their influence for IVARS and Sobol results**. 

The influence is based on how large or small a result is. The lower the ranking the more influential(larger) a factor is.

In [19]:
experiment_1.ivars_factor_ranking

,x1,x2,x3
0.1,1,0,2
0.3,1,0,2
0.5,1,0,2


In [20]:
experiment_1.st_factor_ranking

,x1,x2,x3
,1,0,2


**Bootstrapping results based on the confidence interval limits**

low is the lower limit of the results after `bootstrap_size` sampling iterations

upp is the upper limit of the results after `bootstrap_size` sampling iterations

In [22]:
experiment_1.gammalb

h,x1,x2,x3
0.1,0.003635,0.146042,0.000005
0.2,0.014640,0.593018,0.000015
0.3,0.033075,1.342847,0.000026
0.4,0.058879,2.380803,0.000039
0.5,0.091869,3.674675,0.000052
0.6,0.131740,5.175305,0.000067
0.7,0.178064,6.818604,0.000089
0.8,0.230304,8.528955,0.000122
0.9,0.287815,10.223761,0.000173


In [23]:
experiment_1.gammaub

h,x1,x2,x3
0.1,0.003758,0.152211,0.000009
0.2,0.015141,0.621231,0.000029
0.3,0.034223,1.413181,0.000053
0.4,0.060949,2.515567,0.000078
0.5,0.095129,3.895969,0.000105
0.6,0.136449,5.502404,0.000137
0.7,0.184467,7.265468,0.000181
0.8,0.238619,9.102165,0.000247
0.9,0.298232,10.921149,0.000350


In [24]:
experiment_1.stlb

param,x1,x2,x3
,0.02333,0.944495,0.000014


In [25]:
experiment_1.stub

param,x1,x2,x3
,0.032319,1.28066,0.000032


In [26]:
experiment_1.ivarslb

,x1,x2,x3
0.1,0.000182,0.007302,2.340514e-07
0.3,0.003481,0.141048,3.254933e-06
0.5,0.015616,0.630005,1.101623e-05


In [27]:
experiment_1.ivarsub

,x1,x2,x3
0.1,0.000188,0.007611,4.621551e-07
0.3,0.003601,0.148003,6.484346e-06
0.5,0.016164,0.665017,2.217072e-05


**Reliability estimates of factor rankings based on VARS-TO**

reliability estimates give the ratio of how many bootstrapped rankings were the same as the initial rankings for `bootstrap_size` sampling iterations

In [28]:
experiment_1.rel_st_factor_ranking

,x1,x2,x3
,1.0,1.0,1.0


In [29]:
experiment_1.rel_ivars_factor_ranking

,x1,x2,x3
0.1,1.0,1.0,1.0
0.3,1.0,1.0,1.0
0.5,1.0,1.0,1.0


**Factor grouping based on IVARS50 and Sobol results**

In [30]:
experiment_1.ivars50_grp

,x1,x2,x3
0.5,2,1,2


In [31]:
experiment_1.st_grp

,x1,x2,x3
,2,1,2


**Reliability estimates of rankings based on grouping**

In [32]:
experiment_1.reli_st_grp

,x1,x2,x3
,1.0,1.0,1.0


In [33]:
experiment_1.reli_ivars50_grp

,x1,x2,x3
0.5,1.0,1.0,1.0


#### Advanced Results

**Directional covariogram results**

In [34]:
experiment_1.cov.unstack(0)

param,x1,x2,x3
h,,,
0.1,1.613154,2.096964,1.342891
0.2,1.596263,1.427252,1.343019
0.3,1.577376,0.637124,1.342996
0.4,1.556671,-0.242554,1.342827
0.5,1.534359,-1.175501,1.342515
0.6,1.510678,-2.121429,1.342052
0.7,1.485892,-3.037659,1.341426
0.8,1.460288,-3.880853,1.340618
0.9,1.434173,-4.608808,1.339599


**Directional expected covariogram results**

In [35]:
experiment_1.ecov.unstack(0)

param,x1,x2,x3
h,,,
0.1,0.047890,1.920127,0.000036
0.2,0.031740,1.268328,0.000018
0.3,0.012858,0.478740,0.000003
0.4,-0.008582,-0.418316,-0.000011
0.5,-0.032372,-1.386670,-0.000025
0.6,-0.058271,-2.385709,-0.000039
0.7,-0.086007,-3.371992,-0.000056
0.8,-0.115280,-4.301001,-0.000075
0.9,-0.145765,-5.128965,-0.000099


# DIY

How about you test `experiment_2` yourself? 

In [36]:
experiment_2

Star Centres: 10 Centers Loaded
Star Points: Not Loaded
Parameters: 6 paremeters set
Delta h: 0.1
Model: wavy6D
Seed Number: 123456789
Bootstrap: On
Bootstrap Size: 100
Bootstrap CI: 0.9
Grouping: On
Number of Groups: 2
VARS Analysis: Not Done